In [59]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st
import requests
import json
import gmaps
import os

# Importing Google maps API key
from api_keys import g_key

csv_path = "Criminal_incidents.csv"
csv_df = pd.read_csv(csv_path)

In [60]:
csv_df

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,Unnamed: 9
0,2021,March,Alpine,3691,Dederang,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,1,NaN
1,2021,March,Alpine,3691,Glen Creek,C Drug offences,C30 Drug use and possession,C32 Drug possession,1,NaN
2,2021,March,Alpine,3691,Gundowring,C Drug offences,C10 Drug dealing and trafficking,C12 Drug trafficking,1,NaN
3,2021,March,Alpine,3691,Gundowring,E Justice procedures offences,E10 Justice procedures,E13 Resist or hinder officer,1,NaN
4,2021,March,Alpine,3691,Kergunyah South,B Property and deception offences,B40 Theft,B49 Other theft,1,NaN
...,...,...,...,...,...,...,...,...,...,...
319368,2012,March,Yarriambiack,3487,Lascelles,B Property and deception offences,B50 Deception,B53 Obtain benefit by deception,1,NaN
319369,2012,March,Yarriambiack,3488,Speed,B Property and deception offences,B40 Theft,B49 Other theft,2,NaN
319370,2012,March,Yarriambiack,3489,Tempy,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,1,NaN
319371,2012,March,Yarriambiack,3489,Tempy,A Crimes against the person,"A70 Stalking, harassment and threatening behav...",A732 Non-FV Threatening behaviour,1,NaN


In [61]:
csv_df = csv_df.drop_duplicates(subset="Suburb/Town Name", keep="first")
csv_df = csv_df.reset_index()
csv_df

,index,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,Unnamed: 9
0,0,2021,March,Alpine,3691,Dederang,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,1,NaN
1,1,2021,March,Alpine,3691,Glen Creek,C Drug offences,C30 Drug use and possession,C32 Drug possession,1,NaN
2,2,2021,March,Alpine,3691,Gundowring,C Drug offences,C10 Drug dealing and trafficking,C12 Drug trafficking,1,NaN
3,4,2021,March,Alpine,3691,Kergunyah South,B Property and deception offences,B40 Theft,B49 Other theft,1,NaN
4,5,2021,March,Alpine,3691,Mongans Bridge,B Property and deception offences,B20 Property damage,B21 Criminal damage,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2825,298285,2012,March,Corangamite,3325,Vite Vite North,A Crimes against the person,Other crimes against the person,Other crimes against the person,1,NaN
2826,298788,2012,March,East Gippsland,3875,Tabberabbera,B Property and deception offences,B40 Theft,B42 Steal from a motor vehicle,1,NaN
2827,314553,2012,March,Southern Grampians,3315,Clover Flat,B Property and deception offences,B40 Theft,B49 Other theft,2,NaN
2828,314573,2012,March,Southern Grampians,3315,Tarrayoukyan,B Property and deception offences,B40 Theft,B49 Other theft,1,NaN


In [62]:
suburbs_df = csv_df[["Suburb/Town Name", "Local Government Area"]].copy()
suburbs_df

,Suburb/Town Name,Local Government Area
0,Dederang,Alpine
1,Glen Creek,Alpine
2,Gundowring,Alpine
3,Kergunyah South,Alpine
4,Mongans Bridge,Alpine
...,...,...
2825,Vite Vite North,Corangamite
2826,Tabberabbera,East Gippsland
2827,Clover Flat,Southern Grampians
2828,Tarrayoukyan,Southern Grampians


In [63]:
suburbs_df.rename(columns={"Local Government Area": "LGA"})

,Suburb/Town Name,LGA
0,Dederang,Alpine
1,Glen Creek,Alpine
2,Gundowring,Alpine
3,Kergunyah South,Alpine
4,Mongans Bridge,Alpine
...,...,...
2825,Vite Vite North,Corangamite
2826,Tabberabbera,East Gippsland
2827,Clover Flat,Southern Grampians
2828,Tarrayoukyan,Southern Grampians


In [64]:
suburbs_df["Lat"] = ""
suburbs_df["Lng"] = ""
suburbs_df

,Suburb/Town Name,Local Government Area,Lat,Lng
0,Dederang,Alpine,,
1,Glen Creek,Alpine,,
2,Gundowring,Alpine,,
3,Kergunyah South,Alpine,,
4,Mongans Bridge,Alpine,,
...,...,...,...,...
2825,Vite Vite North,Corangamite,,
2826,Tabberabbera,East Gippsland,,
2827,Clover Flat,Southern Grampians,,
2828,Tarrayoukyan,Southern Grampians,,


In [68]:
suburb_params = {"key": g_key}

for index, row in suburbs_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/geocode/json?"
    
    suburb = row["Suburb/Town Name"]
    
    suburb_params["address"] = f"{suburb}, Vic"
    
    lat_lng = requests.get(base_url, params=suburb_params)
    
    lat_lng = lat_lng.json()
    
    try:
        suburbs_df.loc[index, "Lat"] = lat_lng["results"][0]["geometry"]["location"]["lat"]
        suburbs_df.loc[index, "Lng"] = lat_lng["results"][0]["geometry"]["location"]["lng"]
    
    except:
        suburbs_df.loc[index, "Lat"] = np.nan
        suburbs_df.loc[index, "Lng"] = np.nan

In [69]:
suburbs_df

,Suburb/Town Name,Local Government Area,Lat,Lng
0,Dederang,Alpine,-36.4739,147.018
1,Glen Creek,Alpine,-36.4413,146.936
2,Gundowring,Alpine,-36.348,147.075
3,Kergunyah South,Alpine,-36.4268,147.036
4,Mongans Bridge,Alpine,-36.5897,147.1
...,...,...,...,...
2825,Vite Vite North,Corangamite,-37.771,143.199
2826,Tabberabbera,East Gippsland,-37.548,147.401
2827,Clover Flat,Southern Grampians,-37.5999,141.553
2828,Tarrayoukyan,Southern Grampians,-37.2946,141.615


In [14]:
suburbs_df.to_csv("Suburbs_Lat_lng.csv")